In [1]:
# import relevant modules and files

import numpy as np
import pandas as pd
import mkl
#mkl.set_num_threads(1) # set the number of python threads to be 1 to stop the python parallelisation

df_annotations = {}

for i in range(1, 23):
    df_annotations[i] = pd.read_csv(f"{i}permutationready.tsv", sep="\t")


In [2]:
# !pip install numba
# turns out numba is already installed

In [3]:
annotations = ['3_prime_UTR_variant','5_prime_UTR_variant','NMD_transcript_variant','coding_sequence_variant','downstream_gene_variant','incomplete_terminal_codon_variant','intergenic_variant','intron_variant','mature_miRNA_variant','missense_variant','non_coding_transcript_exon_variant','non_coding_transcript_variant','splice_acceptor_variant','splice_donor_variant','splice_region_variant','start_lost','stop_gained','stop_lost','stop_retained_variant','synonymous_variant','upstream_gene_variant']


In [4]:
# do for all chromosomes
df_noheader = {}
np_df = {}
location_vec = {}
annotation_matrix = {}
a_matrix = {}

for i in range(1,23):
    df_noheader[i] = df_annotations[i].copy()
    np_df[i] = df_noheader[i].to_numpy()
    location_vec[i] = np_df[i][::,1]
    annotation_matrix[i] = np_df[i][::,2:]
    a_matrix[i] = annotation_matrix[i].transpose()
    
    

In [5]:
# Below we try the different parallelisations on a computer with 8 CPU corees, each core 32G of ram
# 1) numba parallelisation (with no numpy threading)
# 2) mp pool multi processing parallelisation (with no numpy threading)




In [6]:
import random
import time
from multiprocessing import Process, Manager
import multiprocessing as mp
import numba

In [7]:
@numba.njit(parallel=True) # cache=True, 

def permutations_nb(num_perms, chr_a_mat, chr_loc_vec, perm_results, perm_per_t, rem, num_threads=1):
    # Parallel using Numba's threading
    for t in numba.prange(num_threads):
        t_perm_start = t * perm_per_t
        t_perm_end = t_perm_start + perm_per_t
        # Give the remainders to the last thread
        if t == num_threads - 1 and rem > 0:
            t_perm_end += rem

        #for i in range(1, 23):
        for perm_id in range(t_perm_start, t_perm_end):
            r = random.randint(0, len(chr_a_mat))
            new_loc = np.roll(chr_loc_vec, r)
            #c_vec = np.matmul(chr_a_mat, new_loc)
            c_vec = np.dot(chr_a_mat, new_loc)
            #c_vec = np.matmul(chr_a_mat, new_loc)
            #count_matrix[i][:, j] = c_vec
            perm_results[:, perm_id] = c_vec


# Number of threads should correspond to the number of CPUs
num_threads = mp.cpu_count()
num_perms = 100000

tic = time.time()
# Numpy array to store all the permutation results
perm_results = np.zeros((23, 21, num_perms))

perms_per_thread, rem = divmod(num_perms, num_threads)

# Run permutations
for chr in range(1, 23):
    permutations_nb(num_perms, a_matrix[chr], location_vec[chr], perm_results[chr], perms_per_thread, rem,
                    num_threads=num_threads)

toc = time.time()
#print(perm_results[2].shape, perm_results[2])
print("Time taken (parallel): {}".format(toc - tic))

#733 seconds for 10k (12.21 mins)
# 7498 seconds for 100k (2hrs)



Time taken (parallel): 7498.256305932999


In [14]:
np.shape(perm_results[1])

(21, 100000)

In [12]:
# save as dataframe

for i in range(1,23):
    pd.DataFrame(perm_results[i]).to_csv(f"{i}_hundred_data.tsv", sep="\t", index=False)

In [18]:
# make a function that counts:
tic = time.time()
def permutations(i): # where i is the number of chromosomes
    count_matrix = {}
    #count_matrix[i] = np.zeros((21,1000)) # alternatively, can change numpy to be column-based from row-based when initialising the matrix, np.zeros((22,21), order='F')
    count_matrix[i] = np.zeros((21, 1000))
    for j in range(0,1000):
        r = random.randint(0, len(location_vec[i+1]))
        new_loc = np.roll(location_vec[i+1], r)
        c_vec = np.matmul(a_matrix[i+1], new_loc)
        count_matrix[i][:,j] = c_vec # each perm stored as row, remember to transpose
    return(count_matrix)
          
process_refs = []
# manager = Manager()
#count_matrix = manager.dict()

from multiprocessing import Pool
with Pool(processes=8) as pool:
    results = pool.map(permutations, [i for i in range(22)])
# remember: 'result' is a list corresponding to the js   
toc = time.time()
time_taken = toc - tic
#print(results)
print("time = {}".format(time_taken))
# for j in range(1, 10):
    # count_matrix[j] = manager.list()
   #  p = Process(target=permutations, args=(j, )) 
    
   # p.start()
   # process_refs.append(p)

#for p in process_refs:
    #p.join()
    
# remember to compare with numpy

time = 33.395068883895874


In [17]:
print(time_taken)

33.099332094192505


In [ ]:
# save as dataframe

for i in range(1,23):
    pd.DataFrame(count_matrix[i]).to_csv(f"{i}_permutation_data.tsv", sep="\t", index=False)

In [18]:
list(count_matrix[1])

[]

In [7]:
count_matrix = {}

tic = time.time()


for i in range(1,23):

    count_matrix[i] = np.zeros((21,1000))
    
    for j in range(0,1000):
        r = random.randint(0, len(location_vec[i]))
        new_loc = np.roll(location_vec[i], r)
        c_vec = np.matmul(a_matrix[i], new_loc)
        count_matrix[i][:,j] = c_vec
    
toc = time.time()
print("Time taken: {}".format(toc-tic))

# took 1902.6005499362946 seconds

Time taken: 190.8539524078369


In [9]:
# save as dataframe

for i in range(1,23):
    pd.DataFrame(count_matrix[i]).to_csv(f"{i}_permutation_data_1000.tsv", sep="\t", index=False)

In [8]:
# save as dataframe

for i in range(1,23):
    pd.DataFrame(count_matrix[i]).to_csv(f"{i}_permutation_data.tsv", sep="\t", index=False)

In [9]:
# calculate original number of sign SNPs in each annotation

original_number = {}

for i in range(1,23):
    original_number[i] = np.matmul(a_matrix[i],location_vec[i])


In [15]:
# graph the data
import matplotlib.pyplot as plt
# for chromosome i, annotation k:
# distribution_matrix[i][k,::] is the values of how many sign SNPs are in that annotation
i = 1
for k in range(0,21):
    plt.figure()
    plt.hist(count_matrix[i][k,::], bins=100)
    plt.gca().set(title='Transcript count Histogram', ylabel='Frequency', xlabel='Number of significant SNPs')
    plt.rcParams['figure.figsize'] = [20, 10]
    plt.axvline(x=original_number[i][k], color='r', label= 'observed number' )
    plt.title('Chromosome ' + str(i) + ' with annotation "' + annotations[k] +'"')
    plt.savefig("figure_"+str(k)+"_chrom_"+str(i))
    plt.close()

In [ ]:
# count_matrix[n] for chromosome n (starts at 1, ends at 22)
# count_matrix[n][k,::] for annotation k (starts at 0, ends at 20)
# count_matrix[n][::,j] for permutation j (starts at 0, ends at 9999)

In [10]:
from collections import Counter

freq_matrix = {} # for all chromosomes

for i in range(1,23):
    
    freq_matrix[i] = {}

    for j in range(0,21):
            freq_matrix[i][j] = Counter(count_matrix[i][j,::]) # count all permutations for one annotation

In [2]:
# freq_matrix[n] is the frequency counter for chromosome n (starts at 1, ends at 22)
# freq_matrix[n][k] is the frequency counter for annotation k, chromosome n (starts at 0, ends at 20)
# freq_matrix[n][k][j] is the number of counts in j 

#freq_matrix[4]

In [11]:
# calculating enrichment
enrichment_matrix = np.zeros((22,21))

for i in range(0,22):
# each row is the count nr in the corresponding chromosome n-1 (so row 0 corresponds to chrom 1)
    for k in range(0,21): # k is the annotation nr
        for l in freq_matrix[i+1][k]: # l is number of sign SNP in annotation
            if l >= original_number[i+1][k]: # and original_number[i+1][k] != 0: 
                enrichment_matrix[i][k] = enrichment_matrix[i][k] + freq_matrix[i+1][k][l] # freq_matrix[i+1][k][l] 
                                                                              # is nr of counts of l sign SNP 
                                                                              # in annotation
                        
p_vals_e = enrichment_matrix/10000 #pvals for enrichment

# find the significant ones

idx = np.where((0 < p_vals_e) & (p_vals_e < 0.05)) # location of significant values

print(idx)
print(p_vals_e[(0 < p_vals_e) & (p_vals_e < 0.05)])

(array([ 0,  0,  0,  0,  1,  1,  1,  1,  2,  3,  3,  3,  6,  6,  6,  8, 10,
       10, 10, 11, 12, 13, 14, 15]), array([ 0,  1,  4, 20,  4,  7, 11, 13, 20,  0,  9, 16,  4, 10, 20,  9,  1,
        4, 20, 20,  1,  4, 16, 13]))
[0.0391 0.0049 0.0277 0.0116 0.0012 0.024  0.0129 0.0342 0.0033 0.0451
 0.0077 0.0153 0.0037 0.005  0.0013 0.0012 0.0087 0.0006 0.0071 0.0329
 0.0433 0.008  0.0266 0.0471]


In [13]:
# Calculating depletion

depletion_matrix = np.zeros((22,21))

for i in range(0,22):
# each row is the count nr in the corresponding chromosome n-1 (so row 0 corresponds to chrom 1)
    for k in range(0,21): # k is the annotation nr
        for l in freq_matrix[i+1][k]: # l is number of sign SNP in annotation
            if l <= original_number[i+1][k]: # and original_number[i+1][k] != 0: 
                depletion_matrix[i][k] = depletion_matrix[i][k] + freq_matrix[i+1][k][l] # freq_matrix[i+1][k][l] 
                                                                              # is nr of counts of l sign SNP 
                                                                              # in annotation

In [16]:
p_vals_d = depletion_matrix/10000 #pvals for enrichment

# find the significant ones

idx2 = np.where((0 < p_vals_d) & (p_vals_d < 0.05)) # location of significant values

print(idx2)
print(p_vals_d[(0 < p_vals_d) & (p_vals_d < 0.05)])

(array([ 0,  1,  2,  5,  8, 10, 13, 15, 16, 17]), array([6, 6, 6, 2, 6, 6, 6, 6, 9, 4]))
[0.0093 0.0001 0.0151 0.0346 0.014  0.0343 0.019  0.0336 0.0439 0.034 ]


In [12]:
# save as dataframe

p_vals_e_df = pd.DataFrame(p_vals_e)

p_vals_e_df.to_csv(f"pvals_enriched.tsv", sep="\t", index=False)

In [15]:
p_vals_d_df = pd.DataFrame(p_vals_d)

p_vals_d_df.to_csv(f"pvals_depleted.tsv", sep="\t", index=False)

In [38]:
np.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/exports/igmm/eddie/GWAS-annotations/annotation-env/lib']
    define_macros = [